In [9]:
import json
from dotenv import load_dotenv
import os

In [10]:
load_dotenv()

True

In [11]:
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")


In [12]:
with open("/Users/iman/Downloads/assets/response_1749827030702_345.json") as f:
    data = json.load(f)
    data_345 = data["data"]

uuids_345 = [entry["uuid"] for entry in data_345]
set_uuids_345 = set(uuids_345)

In [13]:
with open("/Users/iman/Downloads/assets/response_1749825253955_ccbf.json") as f:
    data = json.load(f)
    data_ccbf = data["data"]

uuids_ccbf = [entry["product"]["uuid"] for entry in data_ccbf]
set_uuids_ccbf = set(uuids_ccbf)
original_products_coke = [entry["product"]["baseProduct"] for entry in data_ccbf]

In [14]:
with open("/Users/iman/Downloads/assets/response_1749825091551_telstra.json") as f:
    data = json.load(f)
    data_telstra = data["data"]
uuids_telstra = [entry["product"]["uuid"] for entry in data_telstra]
set_uuids_telstra = set(uuids_telstra)

In [15]:
diff_base_orig = [entry for entry in data if entry["baseProduct"] != entry["originalProduct"]]

TypeError: string indices must be integers, not 'str'

In [16]:
len(diff_base_orig)

NameError: name 'diff_base_orig' is not defined

In [17]:
with open("/Users/iman/Downloads/assets/response_1749825091551_telstra.json") as f:
    data = json.load(f)
    data = data["data"]

uuids_telstra = set([entry["product"]["uuid"] for entry in data])

In [18]:
delta = set_uuids_345 - set_uuids_ccbf - set_uuids_telstra
print(f"Number of unique UUIDs in 345 not in Coke or Telstra: {len(delta)}")

Number of unique UUIDs in 345 not in Coke or Telstra: 609


In [19]:
json.dump(list(delta), open("/Users/iman/Downloads/assets/delta_345.json", "w"))

In [20]:
orgs = {}
orgs_count = {}
for entry in data_ccbf:
    if entry["product"]["organisationUuid"] not in orgs and entry["product"]["organisationUuid"] != '18a7eed1-a257-4aec-88cf-04f213816009':
        orgs[entry["product"]["organisationUuid"]] = {"entries": [], "count": 0}
    if entry["product"]["organisationUuid"] != '18a7eed1-a257-4aec-88cf-04f213816009':
        orgs[entry["product"]["organisationUuid"]]["entries"].append(entry)
        orgs[entry["product"]["organisationUuid"]]["count"] += 1


In [41]:
data = {org: [e["product"]["uuid"] for e in entries["entries"]] for org, entries in orgs.items()}

In [44]:
import pandas as pd
rows = [(org, uuid) for org, uuids in data.items() for uuid in uuids]
df2 = pd.DataFrame(rows, columns=["org", "uuid"])
df2.to_csv("org_inconsistencies.csv")

In [159]:
login_url = "https://api.345.global/api/v1/login/access-token"
credentials = {"username": username, "password": password,"grant_type": "password"}
response = requests.post(login_url, data=credentials)
token_response = response.json()

In [99]:
#access_token = token_response["access_token"]
token_type = token_response.get("token_type", "Bearer") 
token = token_response.get("access_token", None)

In [100]:
# 2. Set headers
headers = {
    "Authorization": f"{token_type} {token}"
}

uuids_filter = ""
from urllib.parse import quote
for entry in delta:
    uuids_filter += f"uuid={entry},"
uuids_filter_url = quote(uuids_filter[:-1])
# 3. Make request to protected endpoint
url = f"https://api.345.global/api/v1/products/paginate?columns=uuid&columns=name&columns=classificationType&columns=organisationUuid&columns=baseProduct&columns=originalProduct&columns=deletedAt&filter=organisationUuid%3D18a7eed1-a257-4aec-88cf-04f213816009%2C{uuids_filter_url}&includeAssets=false&skip=0&limit=5000&withoutDeleted=true&withoutVariants=false&withoutWallpaper=true&withProvisional=false"

response = requests.get(url, headers=headers)


In [205]:
missing_in_ccbf = response_data["data"]

In [209]:
import pandas as pd
df_missing_in_ccbf = pd.DataFrame(missing_in_ccbf)

df_missing_in_ccbf.to_csv("/Users/iman/Downloads/assets/missing_in_ccbf.csv", index=False)

In [198]:
multipack_keywords = ["multipack", "multipacks", "pk","pack","multi","mp"]

In [232]:
[entry["uuid"] for entry in missing_in_ccbf if entry["uuid"] == "8f724755-ebc6-4d97-b2ba-4b673701e551"]

[]

In [200]:
is_multipack = [
    entry for entry in missing_in_ccbf
    if entry.get("name") and any(keyword in entry["name"].lower() for keyword in multipack_keywords)
]
is_single = [entry for entry in missing_in_ccbf
    if entry.get("name") and "single" in entry["name"].lower()
]

In [201]:
len(is_single)

59

In [202]:
len(is_multipack)

346

In [218]:
with open("/Users/iman/Downloads/assets/response_1750174768453_session_19.json") as f:
    data = json.load(f)
    data = data["data"]

uuids_session19 = set([entry["product"]["uuid"] for entry in data])

In [220]:
missing_in_ccbf_uuids = set([entry["uuid"] for entry in missing_in_ccbf])

In [224]:
len(missing_in_ccbf_uuids)

554

In [226]:
uuids_session19 - missing_in_ccbf_uuids

{'8f724755-ebc6-4d97-b2ba-4b673701e551',
 'd730df87-4f01-48db-b52b-417d98e0c7fe'}